In [35]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [34]:
data=pd.read_csv('../output/cleaned_hm.csv')

In [38]:
data.head()

,hmid,wid,reflection_period,original_hm,cleaned_hm,modified,num_sentence,ground_truth_category,predicted_category
0,27673,2053,24h,I went on a successful date with someone I fel...,I went on a successful date with someone I fel...,True,1,NaN,affection
1,27674,2,24h,I was happy when my son got 90% marks in his e...,I was happy when my son got 90% marks in his e...,True,1,NaN,affection
2,27675,1936,24h,I went to the gym this morning and did yoga.,I went to the gym this morning and did yoga.,True,1,NaN,exercise
3,27676,206,24h,We had a serious talk with some friends of our...,We had a serious talk with some friends of our...,True,2,bonding,bonding
4,27677,6227,24h,I went with grandchildren to butterfly display...,I went with grandchildren to butterfly display...,True,1,NaN,affection


In [36]:
import nltk
from nltk.corpus import brown
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import re

In [37]:
from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer
import string
nltk.download("stopwords")
nltk.download("wordnet")
stop = set(stopwords.words('english'))
exclude = set(string.punctuation) 
lemma = WordNetLemmatizer()

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/chenxishi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/chenxishi/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [39]:
def clean(doc):
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    number_free = "".join(ch for ch in punc_free if not ch.isdigit())
    normalized = " ".join(lemma.lemmatize(word) for word in number_free.split())
    normalized_v = " ".join(lemma.lemmatize(word, "v") for word in normalized.split())
    return normalized_v

In [64]:
clean('went to schools')

'go school'

In [49]:
data['stem_hm'] = [clean(senten) for senten in data["cleaned_hm"].tolist()]

In [50]:
content=list(map(lambda x:x.split(),data['stem_hm']))

In [52]:
from gensim import corpora, models
import gensim

In [53]:
dictionary = corpora.Dictionary(content)
corpus = [dictionary.doc2bow(text) for text in content]

In [59]:
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=15, id2word = dictionary, passes=20)


In [60]:
ldamodel.save('../output/lda20pass15topic.model') 

In [62]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
visualization = pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)

/Users/chenxishi/anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


In [63]:
visualization

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
11    -0.250137 -0.111718       1        1  9.749148
14     0.098285 -0.119620       2        1  8.370764
10    -0.130832 -0.185627       3        1  8.284881
2     -0.300206 -0.030748       4        1  6.892705
3     -0.047840  0.199596       5        1  6.839909
7      0.151133 -0.069971       6        1  6.785716
5      0.179650 -0.032163       7        1  6.730416
1      0.126452 -0.075538       8        1  6.701953
0      0.090735 -0.118147       9        1  6.646910
12     0.143617  0.057359      10        1  6.373532
13    -0.060078  0.174028      11        1  6.017565
9      0.118702  0.126144      12        1  5.297966
4     -0.014541 -0.167980      13        1  5.243798
6     -0.057570  0.190400      14        1  5.145634
8     -0.047371  0.163985      15        1  4.919102, topic_info=     Category          Freq        Term         Total  loglift  logprob
term                                                                   
36    Default  16730.000000        make  16730.000000  30.0000  30.0000
37    Default  10701.000000         new  10701.000000  29.0000  29.0000
10    Default  19757.000000       happy  19757.000000  28.0000  28.0000
18    Default  11309.000000      friend  11309.000000  27.0000  27.0000
144   Default  11655.000000        time  11655.000000  26.0000  26.0000
264   Default  11744.000000        work  11744.000000  25.0000  25.0000
3     Default  15773.000000          go  15773.000000  24.0000  24.0000
33    Default   4940.000000       night   4940.000000  23.0000  23.0000
31    Default   7378.000000        last   7378.000000  22.0000  22.0000
9     Default  21751.000000         get  21751.000000  21.0000  21.0000
35    Default   5366.000000        come   5366.000000  20.0000  20.0000
55    Default   4733.000000       watch   4733.000000  19.0000  19.0000
408   Default   4152.000000      moment   4152.000000  18.0000  18.0000
189   Default   4842.000000        home   4842.000000  17.0000  17.0000
334   Default   4056.000000        life   4056.000000  16.0000  16.0000
178   Default   4428.000000       first   4428.000000  15.0000  15.0000
364   Default   4422.000000       month   4422.000000  14.0000  14.0000
985   Default   4307.000000        week   4307.000000  13.0000  13.0000
250   Default   4122.000000      dinner   4122.000000  12.0000  12.0000
312   Default   4394.000000         buy   4394.000000  11.0000  11.0000
365   Default   5598.000000         see   5598.000000  10.0000  10.0000
12    Default   3997.000000         son   3997.000000   9.0000   9.0000
146   Default   5447.000000        able   5447.000000   8.0000   8.0000
88    Default   4765.000000        take   4765.000000   7.0000   7.0000
279   Default   3563.000000         eat   3563.000000   6.0000   6.0000
104   Default   4796.000000        play   4796.000000   5.0000   5.0000
64    Default   4102.000000        feel   4102.000000   4.0000   4.0000
120   Default   4563.000000      family   4563.000000   3.0000   3.0000
232   Default   3132.000000      school   3132.000000   2.0000   2.0000
1402  Default   3240.000000       event   3240.000000   1.0000   1.0000
...       ...           ...         ...           ...      ...      ...
1008  Topic15    475.168549      enough    476.212952   3.0098  -4.5583
1058  Topic15    449.390533      amazon    450.434906   3.0097  -4.6141
2384  Topic15    430.667480       niece    431.711853   3.0096  -4.6566
1002  Topic15    389.813446      amount    390.857819   3.0094  -4.7563
1515  Topic15    388.783875         cup    389.828247   3.0094  -4.7589
1321  Topic15    382.231537      laptop    383.275940   3.0093  -4.7759
3823  Topic15    363.938843    baseball    364.983215   3.0092  -4.8250
352   Topic15    350.354523     account    351.398895   3.0091  -4.8630
1107  Topic15    327.225281  unexpected    328.269684   3.0089  -4.9313
1794  Topic15    318.672119    p